# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [61]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.fs1 import aggregateResults
from ssscoring.fs1 import convertFlySight2SSScoring
from ssscoring.fs1 import dropNonSkydiveDataFrom
from ssscoring.fs1 import getAllSpeedJumpFilesFrom
from ssscoring.fs1 import getSpeedSkydiveFrom
from ssscoring.fs1 import isValidJump
from ssscoring.fs1 import isValidMinimumAltitude
from ssscoring.fs1 import jumpAnalysisTable
from ssscoring.fs1 import processAllJumpFiles
from ssscoring.fs1 import roundedAggregateResults
from ssscoring.fs1 import totalResultsFrom
from ssscoring.fs1 import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [62]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Skydive Arizona | 1509 |
| Skydive Chicago | 616 |
| Thai Sky Adventures | 21 |

In [63]:
dropZoneAltMSL = 23

In [64]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">  23.00 ft</span> (   7.01 m)<h1>')

In [65]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [66]:
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

---
## Results

In [67]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-EC-baselinex,472.446,177.048,295.560,397.872,457.092,475.164,25.0,476.712
09-03-31,440.46,179.352,306.180,399.024,442.404,442.008,25.0,442.440
10-08-57,449.982,174.744,303.696,397.332,439.992,448.380,25.0,452.916
10-58-08,454.014,178.524,310.788,398.448,443.736,452.304,25.0,456.624
11-36-32,456.786,180.396,309.312,400.428,448.884,456.948,25.0,457.704
12-19-48,451.602,175.032,311.580,398.988,442.656,452.736,25.0,452.988
14-00-13,455.616,177.588,302.472,398.232,449.280,453.996,25.0,457.200


### Rounded results for training log

In [68]:
roundedResults = roundedAggregateResults(jumpResults)

### All jumps

In [69]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [70]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [71]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">00-00-00-EC-baselinex jump - 🟢 valid - score = 472.45 km/h</span…

HTML(value='<h3>Max speed = 477; exit at 4237 m (13902 ft), end scoring window at 1981 m (6500 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,177,78,66.2,193,13475
,10.0,296,68,77.0,303,12383
,15.0,398,78,78.8,406,10791
,20.0,457,49,83.9,460,8823
,25.0,475,32,86.1,476,6686


HTML(value='<hr><h1><span style="color: #0f0">09-03-31 jump - 🟢 valid - score = 440.46 km/h</span></h1>')

...

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,175,135,52.3,221,12987
,10.0,312,99,72.3,327,11851
,15.0,399,80,78.6,407,10208
,20.0,443,69,81.1,448,8268
,25.0,453,69,81.3,458,6214


HTML(value='<hr><h1><span style="color: #0f0">14-00-13 jump - 🟢 valid - score = 455.62 km/h</span></h1>')

HTML(value='<h3>Max speed = 457; exit at 4118 m (13510 ft), end scoring window at 1862 m (6109 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,178,137,52.3,225,13084
,10.0,302,94,72.8,317,11965
,15.0,398,71,79.9,404,10340
,20.0,449,32,85.9,450,8340
,25.0,454,6,89.2,454,6266


---
## Summary of all speed skydives in the data lake

In [72]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
00-00-00-EC-baselinex,472,177,296,398,457,475,25.0,477
09-03-31,440,179,306,399,442,442,25.0,442
10-08-57,450,175,304,397,440,448,25.0,453
10-58-08,454,179,311,398,444,452,25.0,457
11-36-32,457,180,309,400,449,457,25.0,458
12-19-48,452,175,312,399,443,453,25.0,453
14-00-13,456,178,302,398,449,454,25.0,457


,totalSpeed,meanSpeed,maxScore
totalSpeed,3180.906,454.415143,472.446
